# Model Fine-tuning Notebook
The purpose of this notebook is exploring and fine-tuning the short listed candidate models to get our best model. In the flow, this notebook covers from step 12 and on.

## Importing Packages

In [ ]:
import sys
import time
from pathlib import Path
import os

import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import root_mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

## Setting up for imports of .py modules

In [ ]:
path = Path(os.getcwd())
path = str(path)
print(path)
sys.path.insert(1, path)

## Importing python modules

In [ ]:
import utils.sml_utils as sml_utils
import utils.regression_utils as reg_utils
import utils.all_attr_eda_utils as aae_utils

## Parameters

In [ ]:
path_to_data = 'data/winequality-white.csv'
target_attr = 'quality'
test_size = 0.20
train_test_split_random_state = 42
missingness_threshold = 0.20

# step 9
elastic_net_random_state = 42
decision_tree_random_state = 42
random_forest_random_state = 42
gradient_boosting_random_state = 42
target_encoder_random_state = 42

# step 10
scoring = ['neg_root_mean_squared_error', 'neg_mean_absolute_error', 'max_error', 'r2']
kwargs = {'return_indices': False}  # if true the indices of the cross validation split are returned
max_iter = 10000  # max number of iterations for Lasso, Ridge and ElasticNet - default was 1000
kfold_n_splits = 10  # number of folds in k-fold cv
kfold_shuffle = True
kfold_random_state = 42

# step 12
gs_cv_kfold_n_splits = 10
gs_cv_kfold_shuffle = True
gs_cv_kfold_random_state = 24
show_all_params = False

## Dataset variables (train/test dataframes were created in previous phases)

In [ ]:
train_df = pd.read_csv('data/wine_train_df.csv').copy() # Make copy so original is not affected
train_cap_x_df = train_df.iloc[:, :-1]  # All columns except the last one
train_y_df = train_df.iloc[:, -1].to_frame()

## Set up time

In [ ]:
start = time.time()

## 1. Checking missingness (also done in previous phases)

In [ ]:
print(df.shape)
df = df.dropna(subset=target_attr)
print(df.shape)

## 2. Train/Test data split (already done in previous phases)

`wine_train_df` and `wine_test_df` were already created back in phase 1

In [ ]:
wine_train_df = pd.read_csv('data/wine_train_df.csv', sep=",")
wine_test_df = pd.read_csv('data/wine_test_df.csv', sep=",")
print(f"wine_train_df shape: {wine_train_df.shape}")
print(f"wine_test_df shape: {wine_test_df.shape}")
del wine_train_df
del wine_test_df

wine_train_df shape: (3918, 12)
wine_test_df shape: (980, 12)


## 3. Train/Validation split

This step will be omitted because cross-validation will be used to select models


## 4. Checking out attribute types
Already completed in previous notebooks.

## 5. Identifying attributes with missingness above threshold
Already completed in previous notebooks.

In [ ]:
return_dict = sml_utils.get_missingness(train_cap_x_df, missingness_threshold)
missingness_drop_list = return_dict['missingness_drop_list']

## 6. Identifying non machine learning attributes
Already completed in previous notebooks.

In [ ]:
non_ml_attr_list = [] # no non-machine learning attributes were identified

## 7. Looking at attributes to exclude from ML
Already completed in previous notebooks.

In [ ]:
ml_attr_drop_list = []

## 8. Establishing ML attribute configuration
Already completed in previous notebooks.

In [ ]:
ml_ignore_list = missingness_drop_list + non_ml_attr_list + ml_attr_drop_list
ml_ignore_list

In [ ]:
# Identifying the remaining numerical attributes to be used

numerical_attr = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',]

# Identifying the remaining nominal attributes to be used

nominal_attr = [] 

assert(train_cap_x_df.shape[1] == len(ml_ignore_list) + len(nominal_attr) + len(numerical_attr))

print(f'ml_ignore_list: {ml_ignore_list}')
print(f'\nnumerical_attr: {numerical_attr}')
print(f'nominal_attr: {nominal_attr}')

print(f'\nnumber of machine learning attributes: {len(numerical_attr) + len(nominal_attr)}')
print(f'\nnumerical_attr and nominal_attr: {numerical_attr + nominal_attr}')

## 9. Building the composite estimators
Shortlist already created in `model_exp_x.ipynb`

In [ ]:
estimator_dict = {
    'LinearRegression': LinearRegression(
        fit_intercept=True, 
        copy_X=True, 
        n_jobs=None, 
        positive=False
    ),

    'ElasticNet': ElasticNet(
        fit_intercept=True,
        copy_X=True, 
        positive=False, 

        alpha=1.0,  
        l1_ratio=0.5,
        max_iter=max_iter, 
        tol=0.0001, 
        selection='cyclic',
        
        precompute=False, 
        warm_start=False, 
        random_state=elastic_net_random_state, 
    ),

    'DecisionTreeRegressor': DecisionTreeRegressor(
        criterion='squared_error', 
        splitter='best', 
        max_depth=None, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        min_weight_fraction_leaf=0.0, 
        max_features=None, 
        random_state=decision_tree_random_state, 
        max_leaf_nodes=None, 
        min_impurity_decrease=0.0, 
        ccp_alpha=0.0, 
        monotonic_cst=None
    ),

    'RandomForestRegressor': RandomForestRegressor(
        
        n_estimators=100,
        bootstrap=True, 
        oob_score=False, 
        n_jobs=None,
        verbose=0, 
        warm_start=False,
        max_samples=None, 
        
        criterion='squared_error',  # in DecisionTreeRegressor
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features=1.0,
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,
        random_state=random_forest_random_state,
        ccp_alpha=0.0,
        monotonic_cst=None
    ),

    'GradientBoostingRegressor': GradientBoostingRegressor(
        
        n_estimators=100, 
        learning_rate=0.1,
        validation_fraction=0.1,  # proportion of training data to set aside as validation set for early stopping
        n_iter_no_change=None,  # used to decide if early stopping will be used to terminate training
        tol=0.0001,  # tolerance for early stopping
        subsample=1.0,  # fraction of samples to be used for fitting the individual base learners
        criterion='friedman_mse',  # what measures quality of split

        loss='squared_error',  # in DecisionTreeRegressor
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features=None,
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,
        random_state=gradient_boosting_random_state,
        ccp_alpha=0.0,
        
        init=None, 
        alpha=0.9, 
        verbose=0, 
        warm_start=False
        
    )
}

### preprocessing pipeline (already done in `model_exp_x.ipynb`)

In [ ]:
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer()),
        ("scaler", StandardScaler())
    ]
)
# discretizer not needed

In [ ]:
# Not used as there are no nominal attributes
nominal_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy='most_frequent')),
        ("ohe", OneHotEncoder())
    ]
)

In [ ]:
preprocessor = ColumnTransformer(
        transformers=[
            ('nominal', nominal_transformer, nominal_attr),
            ('numerical', numerical_transformer, numerical_attr)
        ]
)

In [ ]:
composite_estimator = Pipeline(steps=[('preprocessor', preprocessor), ('estimator', estimator_dict)])

### output of preprocessing step

In [ ]:
trans_train_cap_x_df = pd.DataFrame(
    data=composite_estimator[0].fit_transform(train_cap_x_df),
    index=train_cap_x_df.index,
    columns=[attr_name for attr_name in composite_estimator[0].get_feature_names_out()]
)
trans_train_cap_x_df.head()

,numerical__fixed acidity,numerical__volatile acidity,numerical__citric acid,numerical__residual sugar,numerical__chlorides,numerical__free sulfur dioxide,numerical__total sulfur dioxide,numerical__density,numerical__pH,numerical__sulphates,numerical__alcohol
0,0.515119,-1.076233,0.227731,0.340419,-0.813688,0.534065,-0.641932,-0.447041,-0.328261,-0.702445,1.540371
1,-0.669188,-0.288777,0.895832,1.002071,-0.217212,0.773947,1.355106,0.903370,-0.061886,0.266074,-0.821712
2,-1.498203,0.400248,-0.022807,0.184737,-0.400743,-0.605377,-1.022320,-0.460280,0.404271,0.001933,0.481506
3,0.041396,-0.879369,0.144218,-0.924503,-0.446626,-0.125612,-0.879675,-0.304718,0.137896,0.442168,0.237153
4,0.988842,0.203384,-0.607396,2.432407,0.333383,0.054300,0.855846,1.883079,0.071302,0.089980,-0.088652


## 10. Fitting/evaluating composite estimators
### Surveying the candidate default models by fitting them on the whole train set
This was already completed in `model_exp_x.ipynb`

In [ ]:
return_dict = sml_utils.model_survey_fit(preprocessor, estimator_dict, train_cap_x_df, train_y_df)
trained_estimator_dict = return_dict['trained_estimator_dict']

LinearRegression
ElasticNet
DecisionTreeRegressor
RandomForestRegressor
GradientBoostingRegressor


### estimating the test error rate using k-fold cv (using KFold splitter)

In [ ]:
# get the maximal control k-fold cross validation splitter
splitter = KFold(n_splits=kfold_n_splits, 
                 shuffle=kfold_shuffle, 
                 random_state=kfold_random_state
)

# perform cross validation on models
sml_utils.model_survey_cross_val_and_analysis(preprocessor, estimator_dict, train_cap_x_df, train_y_df, scoring, splitter, target_attr, trained_estimator_dict, **kwargs)

## 11. Shortlisting default models
The shortlist was chosen in `model_exp_x.ipynb`

In [ ]:
estimator_dict.keys()

In [ ]:
# del estimator_dict['INSERT CHOSEN TO DELETE HERE']

# estimator_dict

## 12. Tuning hyperparameters of shortlisted models

### numpy logspace function

In [ ]:
list(np.logspace(-1.1, -0.7, num=5)) # ADJUST THIS AS NEEDED

### numpy arange function

In [ ]:
list(np.arange(0.7, 1.0, step=0.1).round(2)) # ADJUST THIS AS NEEDED

### Setting up hyperparameter space for grid search

In [ ]:
preproc_param_grid = {
    'preprocessor__numerical__imputer__strategy': ['mean', 'median'],
    'preprocessor__nominal__target_encoder__smooth': ['auto']
}

elastic_net_alpha_log_space_min = -1.1
elastic_net_alpha_log_space_max = -0.7
elastic_net_alpha_log_space_num = 5
elastic_net_l1_ratio_log_space_min = 0.7  # l1_ratio = 0 gives Ridge
elastic_net_l1_ratio_log_space_max = 1.0  # l1_ratio = 1 gives Lasso
elastic_net_l1_ratio_log_space_step = 0.1
elastic_net_param_grid = preproc_param_grid | {
    'estimator__alpha': list(np.logspace(
        elastic_net_alpha_log_space_min, 
        elastic_net_alpha_log_space_max, 
        num=elastic_net_alpha_log_space_num)),
    'estimator__l1_ratio': list(np.arange(
        elastic_net_l1_ratio_log_space_min, 
        elastic_net_l1_ratio_log_space_max, 
        step=elastic_net_l1_ratio_log_space_step).round(2))
}

linear_regression_param_grid = preproc_param_grid | {
    'estimator__fit_intercept': [True],  # default value True
    'estimator__copy_X': [True],  # default value True
    'estimator__n_jobs': [None], # default value None
    'estimator__positive': [False]  # default value False
}

decision_tree_param_grid = preproc_param_grid | {
    'estimator__max_depth': [None], # default value None
    'estimator__min_samples_split': [2], # default value 2
    'estimator__min_samples_leaf': [1], # default value 1
    'estimator__max_features': [None], # default value None
    'estimator__max_leaf_nodes': [None] # default value None
}

random_forest_param_grid = preproc_param_grid | {
    
    'estimator__n_estimators': [100],  # default 100
    'estimator__bootstrap': [True],  # default value True
    'estimator__max_samples': [None], # default value None
    
    'estimator__criterion': ['squared_error'],  # default value 'squared_error' for DecisionTreeRegressor
    'estimator__max_depth': [None],  # default value None
    'estimator__min_samples_split': [2],  # default value 2
    'estimator__min_samples_leaf': [1], # default value 1
    'estimator__min_weight_fraction_leaf': [0.0],  # default value 0.0
    'estimator__max_features': [None],  # default value None
    'estimator__max_leaf_nodes': [None],  # default value None
    'estimator__min_impurity_decrease': [0.0],  # default value 0.0
    'estimator__ccp_alpha': [0.0],  # default value 0.0
    'estimator__monotonic_cst': [None]  # default value None
    
}

gradient_boosting_param_grid = preproc_param_grid | {

    'estimator__n_estimators': [100],  # default value 100

    # early stopping
    'estimator__n_iter_no_change': [None],  # default value None
    'estimator__tol': [0.0001],  # default value 0.0001
    'estimator__validation_fraction': [0.1],  # default value 0.1

    # regularization
    'estimator__learning_rate': [0.1],  # default value 0.1
    'estimator__max_depth': [3],  # default value 3
    'estimator__subsample': [1.0],  # default value 1.0
    'estimator__max_features': [None]  # default value None
    
}

param_grids = {
    'LinearRegression': linear_regression_param_grid,
    'ElasticNet': elastic_net_param_grid,
    'DecisionTreeRegressor': decision_tree_param_grid,
    'RandomForestRegressor': random_forest_param_grid,
    'GradientBoostingRegressor': gradient_boosting_param_grid
}

### performing grid search cross validation

In [ ]:
# get the maximal control k-fold cross validation splitter
splitter = KFold(n_splits=gs_cv_kfold_n_splits, 
                 shuffle=gs_cv_kfold_shuffle, 
                 random_state=gs_cv_kfold_random_state
)

# collect grid seach cv results here
tuned_estimator_dict = {}

for estimator_name, estimator in estimator_dict.items():
    
    print(estimator_name)

    composite_estimator = \
    Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('estimator', estimator)
        ]
    )
    
    grid_search_cv = GridSearchCV(
        estimator=composite_estimator, 
        param_grid=param_grids[estimator_name], 
        scoring=scoring,
        n_jobs=None, 
        refit=scoring[0],
        cv=splitter, 
        verbose=0, 
        pre_dispatch='2*n_jobs', 
        error_score=np.nan, 
        return_train_score=True
    )
    grid_search_cv.fit(train_cap_x_df, train_y_df.values.ravel())

    tuned_estimator_dict[estimator_name] = grid_search_cv
    

### flexibility plots

In [ ]:
gs_survey_results_df = pd.DataFrame()
for estimator_name, grid_search_cv in tuned_estimator_dict.items():
    print('\n\n')
    return_dict = sml_utils.plot_flexibility(
        grid_search_cv=grid_search_cv,
        estimator_name=estimator_name,
        scoring=scoring
    )
    results_df = return_dict['results_df']
    results_df['estimator_name'] = estimator_name
    results_df = results_df[['estimator_name'] + [attr for attr in results_df if attr not in ['estimator_name']]]
    gs_survey_results_df = pd.concat([gs_survey_results_df, results_df], axis=0)

gs_survey_results_df = gs_survey_results_df.sort_values(['score', 'best_test_score'])
gs_survey_results_df = gs_survey_results_df.reset_index(drop=True)
gs_survey_results_df

### using cross validation for model selection (TRY ENSEMBLE METHODS??)

In [ ]:
# get the maximal control k-fold cross validation splitter
splitter = KFold(n_splits=kfold_n_splits, 
                 shuffle=kfold_shuffle, 
                 random_state=kfold_random_state
)

# perform cross validation on models
sml_utils.model_tuning_cross_val_and_analysis(tuned_estimator_dict, train_cap_x_df, train_y_df, scoring, splitter, target_attr)

### looking at best hyperparameters

In [ ]:
print('best hyperparameters for each estimator\n')

for index, row in gs_survey_results_df.iterrows():
    
    print(f'\nestimator_name: {row['estimator_name']}; score: {row['score']}')
    
    param_grids_ = param_grids[row['estimator_name']]
    for hyperparameter_name, hyperparameter_value in row['grid_search_cv'].best_params_.items():

        if len(param_grids_[hyperparameter_name]) > 1 and not show_all_params:  #  and only_show_searched_params:  # only check the hyperparameter you are varing
            print(f'   hyperparameter_name: {hyperparameter_name}; hyperparameter_value: {hyperparameter_value}')
        elif show_all_params:
            print(f'   hyperparameter_name: {hyperparameter_name}; hyperparameter_value: {hyperparameter_value}')

In [ ]:
# min test score from GridSeachCV
min_test_score = gs_survey_results_df.best_test_score.min()
print(f'\nMinimum test score from GridSearchCV: {min_test_score}')

# name of estimator with the min test score
estimator_name = gs_survey_results_df.loc[gs_survey_results_df['best_test_score'] == min_test_score, 'estimator_name'].iloc[0]
print(f'\nThe estimator with minimum test score from GridSearchCV is considered the best model. It is: {estimator_name}')

# best estimator
best_model = gs_survey_results_df.loc[gs_survey_results_df['best_test_score'] == min_test_score, 'grid_search_cv'].values[0].best_estimator_

## 13. Evaluating the tuned composite estimators

In [ ]:
rmse_best_model_on_train_set = root_mean_squared_error(train_y_df, best_model.predict(train_cap_x_df))
rmse_best_model_on_train_set
print(f'best_model is the trained estimator that performed the best in GridSearchCV.\n'
      f'\nIt was trained on the whole train set using the hyperparameter combination\n'
      f'that gave the lowest estimate of test error rate in cross validation.\n'
      f'\nThe rmse of the best_model when prediction on the whole train set is {rmse_best_model_on_train_set}.')

## 14. Check for false discoveries
Shuffling the target and using cross validation to see if the discoveries are real or just random chance.

In [ ]:
sml_utils.check_for_false_discoveries(tuned_estimator_dict, train_cap_x_df, train_y_df, scoring, splitter, target_attr, shuffle_target=True,
                                      shuffle_target_random_state=42, gs_survey_results_df=gs_survey_results_df)

## 15. Selecting our model

In [ ]:
composite_estimator = None  # choose best_model here

## 16. Evaluating generalization/making predictions on the test set with our model

In [ ]:
if composite_estimator is None:
    sys.exit('no model was selected')

In [ ]:
test_df = pd.read_csv('data/wine_test_df.csv', index_col='index')
test_df.index.name = None
test_df.head()

In [ ]:
test_cap_x_df = test_df.iloc[:, :-1]
test_cap_x_df.head()

In [ ]:
test_y_df = test_df.iloc[:, -1].to_frame()
test_y_df.head()

In [ ]:
del test_df

In [ ]:
test_rmse = root_mean_squared_error(test_y_df, composite_estimator.predict(test_cap_x_df))
print(f'test_rmse: {test_rmse}')
print(f'relative test_rmse: {test_rmse/np.mean(test_y_df)}')

In [ ]:
pred_y_df = composite_estimator.predict(test_cap_x_df)
data_set_name = 'test'
reg_utils.plot_pred_vs_actual(pred_y_df, test_y_df, data_set_name)

## Script run time

In [ ]:
end = time.time()
print(f'script run time: {(end - start)/60} minutes')